In [ ]:
import streamlit as st
import matplotlib.pyplot as plt


st.set_option('deprecation.showPyplotGlobalUse', False)

def main():

    # Telescope design equations
    def system_focal_length(f1, M):
        return f1*M

    def magnification(q, p):
        return q/p

    def secondary_to_focal_plane_distance(f1, e, M):
        return (f1+e)/(M+1)

    def radius_of_curvature_secondary(p, M):
        return 2*p*M/(M-1)

    def diameter_of_secondary(D1, p, f1):
        return D1*p/f1

    def primary_coefficient(material):
        if material == "Glass":
            return 1.0
        elif material == "Aluminum":
            return 0.96
        elif material == "Ceramic":
            return 0.92
        else:
            return 1.0

    def secondary_coefficient(material):
        if material == "Glass":
            return 1.0
        elif material == "Aluminum":
            return 0.96
        elif material == "Ceramic":
            return 0.92
        else:
            return 1.0

    def resolution(wavelength, D, d):
        return 1.22*wavelength*(D/d)

    def beam_spot_size(wavelength, f1, f2):
        return wavelength*f1*f2/(f1+f2)

    def distance_from_primary(f1, f2, D1, D2):
        return f2*D1/(D2-D1)

    # Streamlit interface
    st.title("Telescope Design Calculator")

    # Get user input
    f1 = st.number_input("Primary mirror focal length (mm)", min_value=0.01, step=0.1)
    e = st.number_input("Distance between primary and secondary mirrors (mm)", min_value=0.001, step=0.1)
    D1 = st.number_input("Primary mirror diameter (mm)", min_value=0.001, step=0.1)
    D2 = st.number_input("Secondary mirror diameter (mm)", min_value=0.001, step=0.1)
    q = st.number_input("Distance from secondary mirror to focal plane (mm)", min_value=0.001, step=0.1)
    M = st.number_input("Magnification of the system", min_value=0.0, step=0.1)
    wavelength = st.number_input("Observation wavelength (nm)", min_value=0.001, step=0.1)
    primary_material = st.selectbox("Primary mirror material", ["Glass", "Aluminum", "Ceramic"])
    secondary_material = st.selectbox("Secondary mirror material", ["Glass", "Aluminum", "Ceramic"])
    lens_type = st.selectbox("Lens type", ["Refractive", "Reflective"])

    #Calculate telescope properties
    p = secondary_to_focal_plane_distance(f1, e, M)
    F = system_focal_length(f1, M)
    R2 = radius_of_curvature_secondary(p, M)
    D2 = diameter_of_secondary(D1, p, f1)

    #include the primary and secondary material in the calculation
    f1 = f1 * primary_coefficient(primary_material)
    e = e * secondary_coefficient(secondary_material)

    #include the wavelength, beam spot and distance
    resolution_telescope = resolution(wavelength, D1, D2)
    beam_spot = beam_spot_size(wavelength, f1, F)
    distance_primary = distance_from_primary(f1, F, D1, D2)

    #show results
    st.write("System focal length: ", F, "mm")
    st.write("Magnification: ", M)
    st.write("Secondary mirror to focal plane distance: ", p, "mm")
    st.write("Radius of curvature of secondary mirror: ", R2, "mm")
    st.write("Diameter of secondary mirror: ", D2, "mm")
    st.write("Primary mirror material: ", primary_material)
    st.write("Secondary mirror material: ", secondary_material)
    st.write("Lens type: ", lens_type)
    st.write("Resolution of the telescope: ", resolution_telescope, "arc sec")
    st.write("Beam spot size: ", beam_spot, "mm")
    st.write("Distance from primary mirror: ", distance_primary, "mm")

    #Create a simple 2D representation of a telescope
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_xlim(-D1/2, D1/2)
    ax.set_ylim(-D1/2, D1/2)
    ax.set_aspect('equal', adjustable='box')
    ax.add_artist(plt.Circle((0,0), D1/2, color='b', fill=False))
    ax.add_artist(plt.Circle((0, distance_primary), D2/2, color='r', fill=False))
    ax.annotate("Primary Mirror", xy=(0, D1/4), xytext=(0, D1/2.5),
                arrowprops=dict(facecolor='b', shrink=0.05))
    ax.annotate("Secondary Mirror", xy=(0, distance_primary + D2/4), xytext=(0, distance_primary + D2/2.5),
                arrowprops=dict(facecolor='r', shrink=0.05))
    ax.plot([0, 0], [0, distance_primary], 'k-')
    #st.pyplot(fig)
    ax.annotate("Beam Spot", xy=(0, distance_primary + beam_spot/4), xytext=(0, distance_primary + beam_spot/2.5),
    arrowprops=dict(facecolor='g', shrink=0.05))
    ax.plot([0, 0], [0, distance_primary], 'k-')
    st.pyplot(fig)



if __name__ == '__main__':
    main()


